# Train YOLOv8 Instance Segmentation on a Custom Dataset



---

[![Roboflow](https://raw.githubusercontent.com/roboflow-ai/notebooks/main/assets/badges/roboflow-blogpost.svg)](https://blog.roboflow.com/how-to-train-yolov8-on-a-custom-dataset)
[![YouTube](https://badges.aleen42.com/src/youtube.svg)](https://youtu.be/wuZtUMEiKWY)
[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/ultralytics/ultralytics)

Ultralytics YOLOv8 is the latest version of the YOLO (You Only Look Once) object detection and image segmentation model developed by Ultralytics. The YOLOv8 model is designed to be fast, accurate, and easy to use, making it an excellent choice for a wide range of object detection and image segmentation tasks. It can be trained on large datasets and is capable of running on a variety of hardware platforms, from CPUs to GPUs.


## Check GPU

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Install Ultralytics for YOLOv8

In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.0.28
!pip install ipython


from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

## YOLO CLI Basics 

If you want to train, validate or run inference on models and don't need to make any modifications to the code, using YOLO command line interface is the easiest way to get started. Read more about CLI in [Ultralytics YOLO Docs](https://docs.ultralytics.com/usage/cli/).

```
yolo task=detect    mode=train    model=yolov8n.yaml      args...
          classify       predict        yolov8n-cls.yaml  args...
          segment        val            yolov8n-seg.yaml  args...
                         export         yolov8n.pt        format=onnx  args...
```

`yolo mode=predict` runs YOLOv8 inference on a variety of sources, downloading models automatically from the latest YOLOv8 release, and saving results to `runs/predict`.

### We used following process to train YOLOv8 model

1. Create project in Roboflow (ArkaSegmentationProject)
2. Upload images
3. Labeling the images (in our case center roof): Use Smart Polygon (Enhanced polygon) to mark the roof
4. Once images are labeled, confirm the annotations and generate new dataset version
5. Export dataset (use yolov8 format to export)

In [ ]:
!pip install roboflow

## Custom Training

### Import dataset from Roboflow


In [ ]:
# Arkasegmentation project download

from roboflow import Roboflow
rf = Roboflow(api_key="9pyHdiM1tENb1DwODp1M")
project = rf.workspace("arka-energy-inc").project("arkasegmentationproject")
dataset = project.version(7).download("yolov8")


> **IMPORTANT**
> 
> Change paths in downloaded dataset data.yaml file to absolute paths. It doesn't work otherwise!

In [ ]:
print(dataset.location)

#### Use YOLOv8 model 'm'

In [ ]:
%cd {HOME}

!yolo task=segment mode=train model=yolov8m-seg.pt data={dataset.location}/data.yaml epochs=100 imgsz=640 plots=True

#### Use the directory that is shown at the end of the training to see training results

In [ ]:
!ls {HOME}/runs/segment/train6/

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/segment/train6/confusion_matrix.png', width=600)


In [ ]:

Image(filename=f'{HOME}/runs/segment/train6/MaskPR_curve.png', width=600)


In [ ]:

Image(filename=f'{HOME}/runs/segment/train6/MaskF1_curve.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/segment/train6/results.png', width=600)

### Validate Custom Model

In [ ]:
%cd {HOME}

!yolo task=segment mode=val model={HOME}/runs/segment/train6/weights/best.pt data={dataset.location}/data.yaml

### Inference with Custom Model

In [ ]:
%cd {HOME}
print({dataset.location})
!yolo task=segment mode=predict model={HOME}/runs/segment/train6/weights/best.pt conf=0.50 source={dataset.location}/test/images save=True

**NOTE:** Let's take a look at few results.

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/segment/predict2/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")

In [ ]:
#Run inference on your model on a persistant, auto-scaling, cloud API

#load model
model = project.version(dataset.version).model

#choose random test set image
import os, random
test_set_loc = dataset.location + "/test/images/"
print(test_set_loc)
random_test_image = random.choice(os.listdir(test_set_loc))
print("running inference on " + random_test_image)

#pred = model.predict(test_set_loc + random_test_image, confidence=40, overlap=30).json()
#pred = model.predict(test_set_loc + random_test_image, confidence=40).json()
pred = model.predict(test_set_loc + random_test_image, confidence=40).save("test.jpg")
pred

## Deploy model on Roboflow

Once you have finished training your YOLOv8 model, you’ll have a set of trained weights ready for use. These weights will be in the `/runs/detect/train/weights/best.pt` folder of your project. You can upload your model weights to Roboflow Deploy to use your trained weights on our infinitely scalable infrastructure.

The `.deploy()` function in the [Roboflow pip package](https://docs.roboflow.com/python) now supports uploading YOLOv8 weights.

To upload model weights, add the following code to the “Inference with Custom Model” section in the aforementioned notebook:

In [ ]:
!pip install ultralytics==8.0.134

#### Deploy the trained model

In [ ]:
project.version(dataset.version).deploy(model_type="yolov8-seg", model_path=f"{HOME}/runs/segment/train6/")


#### Check that the deployment is working

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="9pyHdiM1tENb1DwODp1M")
project = rf.workspace("arka-energy-inc").project("arkasegmentationproject")
model = project.version(4).model

# infer on a local image
print(model.predict("ArkaCA_401.png", confidence=40).json())

# visualize your prediction
model.predict("your_image.jpg", confidence=40, overlap=30).save("prediction.jpg")

# infer on an image hosted elsewhere
# print(model.predict("URL_OF_YOUR_IMAGE", hosted=True, confidence=40, overlap=30).json())